# Preparación Examen Final

__Autor:__ Lucía Saiz Lapique

In [1]:
from pyspark import SparkContext
sc = SparkContext("local", "First App")

In [2]:
data_file1 = "./fight_data.csv"  # direccion en el disco duro 
combates = sc.textFile(data_file1)

In [3]:
combates.take(1)

['R_fighter;B_fighter;win_by;last_round;last_round_time;Format;Referee;date;location;Winner;weight_class;title_bout']

In [4]:
combates = combates.map(lambda x: x.split(";"))

In [5]:
combates.take(3)

[['R_fighter',
  'B_fighter',
  'win_by',
  'last_round',
  'last_round_time',
  'Format',
  'Referee',
  'date',
  'location',
  'Winner',
  'weight_class',
  'title_bout'],
 ['Henry Cejudo',
  'Marlon Moraes',
  'KO/TKO',
  '3',
  '4:51',
  '5 Rnd (5-5-5-5-5)',
  'Marc Goddard',
  'June 08, 2019',
  'Chicago, Illinois, USA',
  'Henry Cejudo',
  'Bantamweight',
  'True'],
 ['Valentina Shevchenko',
  'Jessica Eye',
  'KO/TKO',
  '2',
  '0:26',
  '5 Rnd (5-5-5-5-5)',
  'Robert Madrigal',
  'June 08, 2019',
  'Chicago, Illinois, USA',
  'Valentina Shevchenko',
  "Women's Flyweight",
  'True']]

In [6]:
combates = combates.filter(lambda x: 'R_fighter' not in x)
combates.cache()

PythonRDD[4] at RDD at PythonRDD.scala:48

In [7]:
combates.take(1)

[['Henry Cejudo',
  'Marlon Moraes',
  'KO/TKO',
  '3',
  '4:51',
  '5 Rnd (5-5-5-5-5)',
  'Marc Goddard',
  'June 08, 2019',
  'Chicago, Illinois, USA',
  'Henry Cejudo',
  'Bantamweight',
  'True']]

### 1. ¿Qué luchador ha luchado más veces con calzón azul?

In [8]:
## meter contador cuando me pida algo relacionado con la cantidad que hay de alguna variable 
mas_veces = combates.map(lambda x: (x[1], 1))
mas_veces_key = mas_veces.reduceByKey(lambda x, y: x + y)
mas_veces_key.take(3)

[('Paul Sass', 2), ('Sean Sherk', 4), ('Anthony Smith', 10)]

In [9]:
mas_veces_key.max(lambda x: x[1])

('Jeremy Stephens', 19)

In [10]:
mas_veces_key.sortBy(lambda x: x[1], ascending = False).take(3)

[('Jeremy Stephens', 19), ('Charles Oliveira', 17), ('Nik Lentz', 14)]

In [11]:
mas_veces_key.sortBy(lambda x: -x[1]).take(3)

[('Jeremy Stephens', 19), ('Charles Oliveira', 17), ('Nik Lentz', 14)]

### Arbitro que ha dirigido más combates por año

In [12]:
## igual que el anterior
combates.map(lambda x: (x[6], 1)).reduceByKey(lambda x, y: x + y).max(lambda x: x[1])

('Herb Dean', 726)

### Número de combates por año (hay que trabajar sobre la fecha)

In [13]:
year = combates.map(lambda x: (x[7][-4:], 1))
year.take(3)

[('2019', 1), ('2019', 1), ('2019', 1)]

In [14]:
year.reduceByKey(lambda x, y: x + y).sortBy(lambda x: x[0]).collect()

[('1993', 8),
 ('1994', 29),
 ('1995', 33),
 ('1996', 36),
 ('1997', 39),
 ('1998', 20),
 ('1999', 43),
 ('2000', 41),
 ('2001', 40),
 ('2002', 53),
 ('2003', 41),
 ('2004', 39),
 ('2005', 80),
 ('2006', 158),
 ('2007', 171),
 ('2008', 201),
 ('2009', 215),
 ('2010', 253),
 ('2011', 300),
 ('2012', 341),
 ('2013', 386),
 ('2014', 503),
 ('2015', 473),
 ('2016', 493),
 ('2017', 451),
 ('2018', 474),
 ('2019', 223)]

### Por año y por categoría de peso, lo q pone en la pizarra

In [15]:
combates1 = combates.filter(lambda x: 'No Time Limit' not in x)
solucion = combates1.map(lambda x: ((x[7][-4:], x[10]), (int(x[3])/int(x[5][0]))))
cantidad = combates1.map(lambda x: ((x[7][-4:], x[10]), 1))
solucion.take(3)

[(('2019', 'Bantamweight'), 0.6),
 (('2019', "Women's Flyweight"), 0.4),
 (('2019', 'Lightweight'), 0.6666666666666666)]

In [16]:
cantidad_key = cantidad.reduceByKey(lambda x, y: x + y)
solucion_key = solucion.reduceByKey(lambda x, y: x + y)

In [17]:
resultado = solucion_key.join(cantidad_key)
resultado = resultado.map(lambda x: (x[0], (x[1][0]/x[1][1])))

In [18]:
resultado.take(5)

[(('2001', 'Welterweight'), 0.7407407407407407),
 (('2016', 'Catch Weight'), 0.5111111111111112),
 (('1999', 'Open Weight'), 0.8333333333333333),
 (('2011', 'Lightweight'), 0.7746031746031745),
 (('2017', 'Light Heavyweight'), 0.6974358974358973)]

### ¿Cómo se han ganado más, de azul o de rojo?

In [19]:
rojos_ganados = combates.filter(lambda x: x[0] == x[9]).map(lambda x: x[9])
rojos_ganados.take(5)

['Henry Cejudo',
 'Valentina Shevchenko',
 'Tony Ferguson',
 'Tatiana Suarez',
 'Aljamain Sterling']

In [20]:
azules_ganados = combates.filter(lambda x: x[1] == x[9]).map(lambda x: x[9])
azules_ganados.take(5)

['Petr Yan',
 'Blagoy Ivanov',
 'Alexa Grasso',
 'Calvin Kattar',
 'Darren Stewart']

In [21]:
azules_ganados.count()

1591

In [22]:
rojos_ganados.count()

3470

### ¿Cuántos combates han celebrado el día 5 o 15 del mes en Chicago?

In [23]:
prueba = combates.filter(lambda x: x[7][-8:-6] == '23' or x[7][-8:-6] == '09').map(lambda x: (x[7], x[8]))
prueba.take(5)

[('March 23, 2019', 'Nashville, Tennessee, USA'),
 ('March 23, 2019', 'Nashville, Tennessee, USA'),
 ('March 23, 2019', 'Nashville, Tennessee, USA'),
 ('March 23, 2019', 'Nashville, Tennessee, USA'),
 ('March 23, 2019', 'Nashville, Tennessee, USA')]

In [24]:
dos = prueba.filter(lambda x: x[1] == 'Chicago, Illinois, USA')
dos.count()

24

In [25]:
prueba.filter(lambda x: 'Chicago' in x[1]).count()  # otra forma de hacerlo

24

In [26]:
data_file2 = "./fighter_details.csv"  # direccion en el disco duro 
luchadores = sc.textFile(data_file2)

## Con SQL

In [27]:
from pyspark.sql import Row, SQLContext  # siempre tal cual
from pyspark.sql.functions import *
sqlContext = SQLContext(sc)

In [28]:
combates_sql = combates.map(lambda p: Row( # para facilitar el uso de las variables, no es necesario
    rojo = p[0],
    azul = p[1],
    year = int(p[7][-4:]),
    winner = p[9],
    categoria = p[10]
))

In [29]:
combates_df = sqlContext.createDataFrame(combates_sql)  # siempre tal cual
combates_df.registerTempTable('combates_final')

In [30]:
#sqlContext.sql("""   

#""").show()   ##siempre tal cual

In [31]:
sqlContext.sql("""   
    SELECT DISTINCT winner
    FROM combates_final
    ORDER BY winner
    LIMIT 10
""").show()

+--------------------+
|              winner|
+--------------------+
|                    |
|         Aaron Riley|
|          Aaron Rosa|
|       Aaron Simpson|
|Abdul Razak Alhassan|
|  Abdul-Kerim Edilov|
|       Abel Trujillo|
|         Abu Azaitar|
|       Adam Milstead|
|      Adam Wieczorek|
+--------------------+



### Número de combates por año y por categoría

In [32]:
sqlContext.sql("""   
    SELECT year, categoria, COUNT(*)
    FROM combates_final
    WHERE year BETWEEN 1993 AND 1997
    GROUP BY year, categoria
    ORDER BY year 
""").show()

+----+-----------------+--------+
|year|        categoria|count(1)|
+----+-----------------+--------+
|1993|     Catch Weight|       1|
|1993|      Open Weight|       7|
|1994|     Catch Weight|       3|
|1994|      Open Weight|      26|
|1995|      Open Weight|      26|
|1995|     Catch Weight|       7|
|1996|     Catch Weight|       5|
|1996|      Open Weight|      31|
|1997|      Lightweight|      11|
|1997|     Middleweight|       1|
|1997|      Heavyweight|      26|
|1997|Light Heavyweight|       1|
+----+-----------------+--------+



In [33]:
## para pasar datos de formato sql a rdd

prueba_rdd = sqlContext.sql("""
    SELECT year, categoria, count(*)
    FROM combates_final
    WHERE year BETWEEN 1993 AND 1997
    GROUP BY year, categoria
    ORDER BY year
""")
prueba_rdd.show()
rdd_mio = prueba_rdd.rdd
rdd_mio.take(5)
jota = rdd_mio.map(lambda x: x.categoria)
jota.take(5)

+----+-----------------+--------+
|year|        categoria|count(1)|
+----+-----------------+--------+
|1993|     Catch Weight|       1|
|1993|      Open Weight|       7|
|1994|     Catch Weight|       3|
|1994|      Open Weight|      26|
|1995|      Open Weight|      26|
|1995|     Catch Weight|       7|
|1996|     Catch Weight|       5|
|1996|      Open Weight|      31|
|1997|      Lightweight|      11|
|1997|     Middleweight|       1|
|1997|      Heavyweight|      26|
|1997|Light Heavyweight|       1|
+----+-----------------+--------+



['Catch Weight', 'Open Weight', 'Catch Weight', 'Open Weight', 'Open Weight']

In [34]:
sc.stop()